In [1]:
import pandas as pd
import numpy as np

import dendropy

import os

import pysam
import vamb

import glob

import pickle

# Purpose of this Notebook

In order to test the relevant impact of tuning the beta parameter for out disentangled autoencoder, I want to set up some compelling experiments. These will consist of a small grid search methodology where we modify both the number of genomes in a simulation as well as the phylogenetic distance between those genomes. That strategy would look something like this:



|     | 2   |  10 | 50  |
|-----|-----|-----|-----|
| 5   |     |     |     |
| 50  |     |     |     |
| 250 |     |     |     |


In order do do this, I'm going to set up these simulations first by deciding how I'll collect input reference genomes of varying levels of phylogenetic similarity.

## 1. Gathering Genomes and Adjusting Similarity

We'll start by obtaining the complete list of bacterial genomes present in the NCBI Genome database:

In [3]:
!wget ftp://ftp.ncbi.nlm.nih.gov/genomes/genbank/bacteria/assembly_summary.txt

--2021-03-14 00:32:06--  ftp://ftp.ncbi.nlm.nih.gov/genomes/genbank/bacteria/assembly_summary.txt
           => ‘assembly_summary.txt’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.13, 130.14.250.12, 2607:f220:41e:250::10, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.13|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /genomes/genbank/bacteria ... done.
==> SIZE assembly_summary.txt ... 276853487
==> PASV ... done.    ==> RETR assembly_summary.txt ... done.
Length: 276853487 (264M) (unauthoritative)

assembly_summary.tx 100%[===================>] 264.03M  23.2MB/s    in 12s     

2021-03-14 00:32:19 (21.4 MB/s) - ‘assembly_summary.txt’ saved [276853487]



In [4]:
all_ncbi_genomes = pd.read_csv('assembly_summary.txt', sep='\t', skiprows=1)
all_complete_genomes = all_ncbi_genomes
#all_complete_genomes = all_ncbi_genomes[all_ncbi_genomes['assembly_level']=='Complete Genome']
all_complete_genomes = all_complete_genomes[(~all_complete_genomes['organism_name'].str.contains('sp.')) & 
                                            (~all_complete_genomes['organism_name'].str.contains('endosymbiont')) & 
                                            (~all_complete_genomes['organism_name'].str.contains('candidate'))]
all_complete_genomes = all_complete_genomes[pd.notna(all_complete_genomes['excluded_from_refseq'])]

all_complete_genomes = all_complete_genomes[['# assembly_accession','wgs_master','taxid','organism_name',
                                             'infraspecific_name','ftp_path']].drop_duplicates()

all_complete_genomes = all_complete_genomes.groupby('organism_name').first().reset_index()

/home/pathinformatics/miniconda3/envs/vamb_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
all_complete_genomes.head(2)

,organism_name,# assembly_accession,wgs_master,taxid,infraspecific_name,ftp_path
0,'Brassica napus' phytoplasma,GCA_003181115.1,QGKT00000000.1,469009,None,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/003...
1,'Candidatus Kapabacteria' thiocyanatum,GCA_001899175.1,MKVH00000000.1,1895771,None,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/001...


Having this, I've also obtained a reference phylogenetic tree which we can use to group genome by differing levels of phylogenetic similarity. The particular tree I'll be using is one we obtained from the [Web Of Life (WOL)](https://biocore.github.io/wol/start) project. I read this in with dendropy and then convert this into a phylogenetic distance matrix `pdm`. This is calculated using the patristic distances between items in the tree and, thus, necessitates a pairwise calculation which can take quite a bit. Moreover, this creates a pretty big pickle file (several GB).

In [6]:
# with open('astral.cons.nid.e5p50.nwk', 'r') as treefile:
#     tree = dendropy.Tree.get(
#             data=treefile.read(),
#             schema="newick",
#             )
    
#     pdm = tree.phylogenetic_distance_matrix()

In [7]:
#with open('bacteria_pdm.pickle', 'wb') as f:
#    pickle.dump(pdm, f, pickle.HIGHEST_PROTOCOL)

I will then use this distance matrix to grab a collection of genomes relevant to each of our experiments. This will be a collection of genomes from 1 genus, from 5 genera, and from 10 genera. I'll first select which genera these are and then I'll create three specimens for each genera set: one with 25 genomes, one with 50 genomes, and one with 250 genomes genomes.

So we start with our genera list which represent both common bacterial genera and things that are not totally unique giving us the ability to demonstrate similar vs un-similar genomes.

In [8]:
genus_set_1 = 'Escherichia'
genus_set_2, genus_set_3, genus_set_4, genus_set_5 = 'Vibrio', 'Bacteroides', 'Clostridium', 'Listeria'
genus_set_6, genus_set_7, genus_set_8, genus_set_9, genus_set_10 = 'Lactobacillus', 'Staphylococcus', 'Shigella', 'Pseudomonas', 'Bifidobacterium'

So finally, I can use these genera labels to create the actual genome lists to use for sample simulation.

In [13]:
for idx, genus_set in enumerate([[genus_set_1], 
                  [genus_set_1, genus_set_2, genus_set_3, genus_set_4, genus_set_5], 
                  [genus_set_1, genus_set_2, genus_set_3, genus_set_4, genus_set_5, genus_set_6, genus_set_7, genus_set_8, genus_set_9, genus_set_10]]):
    
    for num_genomes in [25, 50, 250]:
        if idx == 0:
            num_genera = 1
        else:
            num_genera = idx*5
        complexity_sim_folder = os.path.join(os.getcwd(), f"example_input_data/new_simulations/complexity_sim_{num_genera}_genera_{num_genomes}_genomes")
        if not os.path.exists(complexity_sim_folder):
            os.mkdir(complexity_sim_folder)
            os.mkdir(os.path.join(complexity_sim_folder, 'genomes'))

            for genus in genus_set:
                genus_subset = all_complete_genomes[all_complete_genomes['organism_name'].str.startswith(genus)]

                per_genus_sample_size = num_genomes/len(genus_set)
                genomes = genus_subset.sample(int(per_genus_sample_size))
                print(genus, per_genus_sample_size, genomes['ftp_path'].nunique())

                for ftp_path in genomes['ftp_path'].values:
                    genome_path = ftp_path + f"/{os.path.basename(ftp_path)}_genomic.fna.gz"
                    genome_folder = os.path.join(complexity_sim_folder, 'genomes')
                    !wget $genome_path -P $genome_folder

Escherichia 2.5 2
--2021-03-14 01:23:23--  ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/000/397/385/GCA_000397385.1_Ec13_E2F4_TOP293-4_120714/GCA_000397385.1_Ec13_E2F4_TOP293-4_120714_genomic.fna.gz
           => ‘/home/pathinformatics/jupyter_projects/vamb/stanford_cs230_project/example_input_data/new_simulations/complexity_sim_10_genera_25_genomes/genomes/GCA_000397385.1_Ec13_E2F4_TOP293-4_120714_genomic.fna.gz’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.12, 130.14.250.13, 2607:f220:41e:250::10, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.12|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /genomes/all/GCA/000/397/385/GCA_000397385.1_Ec13_E2F4_TOP293-4_120714 ... done.
==> SIZE GCA_000397385.1_Ec13_E2F4_TOP293-4_120714_genomic.fna.gz ... 1626284
==> PASV ... done.    ==> RETR GCA_000397385.1_Ec13_E2F4_TOP293-4_120714_genomic.fna.gz ... done.
Length: 16

I'll also just verify that we got the right number of genomes downloaded and the right number of genera represented in each of these experiments.

In [14]:
for idx, genus_set in enumerate([[genus_set_1], 
                  [genus_set_1, genus_set_2, genus_set_3, genus_set_4, genus_set_5], 
                  [genus_set_1, genus_set_2, genus_set_3, genus_set_4, genus_set_5, genus_set_6, genus_set_7, genus_set_8, genus_set_9, genus_set_10]]):
    
    for num_genomes in [25, 50, 250]:
        param_thing = f"{len(genus_set)}_genera_{num_genomes}_genomes"
        print(param_thing)

        print(len(glob.glob(f"example_input_data/new_simulations/complexity_sim_{param_thing}/genomes/*")) )

        targets = ['_'.join(i.split('/')[-1].strip('.fna.gz').split('_')[:2]) for 
                   i in glob.glob(f"example_input_data/new_simulations/complexity_sim_{param_thing}/genomes/*")]
        
        assert len(targets) == num_genomes

        all_complete_genomes[all_complete_genomes['# assembly_accession'].isin(targets)]['organism_name'].str.contains(genus_set_1).all()
        
        assert len(set(genus_set).intersection(
            set([i.split(' ')[0] for 
                 i in all_complete_genomes[all_complete_genomes['# assembly_accession'].isin(targets)]['organism_name'].values]))) == len(genus_set)    

1_genera_25_genomes
25
1_genera_50_genomes
50
1_genera_250_genomes
250
5_genera_25_genomes
25
5_genera_50_genomes
50
5_genera_250_genomes
250
10_genera_25_genomes
25
10_genera_50_genomes
50
10_genera_250_genomes
250


We also have to set up the genomes and metadata tsv files for simulation

In [ ]:
for complexity_experiment in glob.glob('example_input_data/new_simulations/complexity*'):
    experiment_name = os.path.basename(complexity_experiment)
    
    for fna_file in glob.glob(f"example_input_data/new_simulations/{experiment_name}/genomes/*.gz"):
        !gzip -d $fna_file
    
    genomes = glob.glob(os.path.join(complexity_experiment,'genomes','*'))
    genomes_mod = [os.path.join('/input/genomes',os.path.basename(i)) for i in genomes]
    
    df = pd.DataFrame(genomes_mod)
    df['genome_ID'] = [ f"Genome{float(i+1)}" for i in range(len(df))]
    df = df[['genome_ID',0]]
    df.to_csv(os.path.join(complexity_experiment,'genome_to_id.tsv'), sep='\t', header=False, index=False)
    
    metadata_df = df
    metadata_df['novelty_category'] = 'known_strain'
    metadata_df['NCBI_ID'] = metadata_df[0].str.split('/', expand=True)[3].str.split('_', expand=True)[[0,1]].agg('_'.join, axis=1)
    
    metadata_df = metadata_df.merge(all_complete_genomes, left_on='NCBI_ID', 
                      right_on='# assembly_accession', 
                      how='inner')
    metadata_df['NCBI_ID'] = metadata_df['ftp_path'].str.split('/', expand=True)[9].str.strip('.fna.gz')
    
    metadata_df = metadata_df[['genome_ID','NCBI_ID','taxid','novelty_category']].rename(columns={'NCBI_ID':'OTU','taxid':'NCBI_ID',})
    metadata_df['NCBI_ID'] = 2
    
    metadata_df.to_csv(os.path.join(complexity_experiment,'metadata.tsv'), sep='\t', index=False)

And now I can run this to create the simulated samples

In [301]:
experiments = [os.path.basename(i) for i in glob.glob('example_input_data/new_simulations/complexity*')]

for experiment in experiments:
    if len(glob.glob(os.path.join('example_input_data/new_simulations',experiment,'*','reads','anonymous*'))) == 0:
        print('simulating',experiment)
        
        input_folder = os.path.join(os.getcwd(), f"example_input_data/new_simulations/{experiment}")
        output_folder = os.path.join(os.getcwd(), f"example_input_data/new_simulations/{experiment}")

        input_directory = f"{input_folder}:/input:rw"
        output_directory = f"{output_folder}:/output:rw"

        !echo awhf98aq | sudo -S docker run --rm -v $input_directory -v $output_directory  \
            cami/camisim:latest  metagenomesimulation.py /input/mini_config.ini

simulating complexity_sim_1_genera_250_genomes
Password:2021-03-13 09:38:14 INFO: [MetagenomeSimulationPipeline] Metagenome simulation starting
2021-03-13 09:38:14 INFO: [MetagenomeSimulationPipeline] Validating Genomes
2021-03-13 09:38:14 INFO: [MetadataReader] Reading file: '/input/genome_to_id.tsv'
2021-03-13 09:43:56 INFO: [MetagenomeSimulationPipeline] Design Communities
2021-03-13 09:43:56 INFO: [CommunityDesign] Drawing strains.
2021-03-13 09:43:56 INFO: [MetadataReader 82380310288] Reading file: '/input/metadata.tsv'
2021-03-13 09:43:56 INFO: [MetadataReader 28782960663] Reading file: '/input/genome_to_id.tsv'
2021-03-13 09:43:56 INFO: [CommunityDesign] Validating raw sequence files!
2021-03-13 09:44:34 INFO: [NcbiTaxonomy] Building taxonomy tree...
2021-03-13 09:44:34 INFO: [NcbiTaxonomy] Reading 'nodes' file:	'/tmp/tmpHCLv64/NCBI/nodes.dmp'
2021-03-13 09:44:50 INFO: [NcbiTaxonomy] Reading 'names' file:	'/tmp/tmpHCLv64/NCBI/names.dmp'
2021-03-13 09:44:53 INFO: [NcbiTaxonomy] R

In [311]:
glob.glob('example_input_data/new_simulations/com*/*sample_0*/reads/*anonymous_reads*')

['example_input_data/new_simulations/complexity_sim_1_genera_250_genomes/2021.03.13_09.38.14_sample_0/reads/anonymous_reads.fq.gz',
 'example_input_data/new_simulations/complexity_sim_1_genera_25_genomes/2021.03.13_09.21.48_sample_0/reads/anonymous_reads.fq.gz',
 'example_input_data/new_simulations/complexity_sim_5_genera_25_genomes/2021.03.13_09.51.05_sample_0/reads/anonymous_reads.fq.gz',
 'example_input_data/new_simulations/complexity_sim_10_genera_25_genomes/2021.03.13_09.59.34_sample_0/reads/anonymous_reads.fq.gz',
 'example_input_data/new_simulations/complexity_sim_1_genera_50_genomes/2021.03.13_10.06.07_sample_0/reads/anonymous_reads.fq.gz',
 'example_input_data/new_simulations/complexity_sim_10_genera_50_genomes/2021.03.13_10.16.05_sample_0/reads/anonymous_reads.fq.gz',
 'example_input_data/new_simulations/complexity_sim_5_genera_50_genomes/2021.03.13_10.23.50_sample_0/reads/anonymous_reads.fq.gz',
 'example_input_data/new_simulations/complexity_sim_5_genera_250_genomes/2021.03

# Now we just need to run the assemblies

Note that many of these steps are described in greater detail in the `preparing_data` notebook.

In [365]:
BASE_DIR = os.getcwd()

In [417]:
for SIM_FASTA_FILE in glob.glob('example_input_data/new_simulations/complexity_sim*/*sample_0*'):
    input_contigs_fasta = os.path.join(BASE_DIR, f"{SIM_FASTA_FILE}/contigs/anonymous_gsa.fasta.gz")
    output_filtered_contigs_fasta = os.path.join(BASE_DIR, f"{SIM_FASTA_FILE}/contigs/anonymous_gsa_filtered.fasta.gz")

    with vamb.vambtools.Reader(input_contigs_fasta, 'rb') as inputfile:
        with open(output_filtered_contigs_fasta, 'w') as outputfile:
            print(inputfile)
            print(outputfile)
            vamb.vambtools.filtercontigs(inputfile, outputfile, minlength=2000)

<_io.TextIOWrapper name='/Users/pmccaffrey/jupyter_projects/CS230_Stanford_Project/example_input_data/new_simulations/complexity_sim_1_genera_250_genomes/2021.03.13_09.38.14_sample_0/contigs/anonymous_gsa_filtered.fasta.gz' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='/Users/pmccaffrey/jupyter_projects/CS230_Stanford_Project/example_input_data/new_simulations/complexity_sim_1_genera_25_genomes/2021.03.13_09.21.48_sample_0/contigs/anonymous_gsa_filtered.fasta.gz' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='/Users/pmccaffrey/jupyter_projects/CS230_Stanford_Project/example_input_data/new_simulations/complexity_sim_5_genera_25_genomes/2021.03.13_09.51.05_sample_0/contigs/anonymous_gsa_filtered.fasta.gz' mode='w' encoding='UTF-8'>
<_io.TextIOWrapper name='/Users/pmccaffrey/jupyter_projects/CS230_Stanford_Project/example_input_data/new_simulations/complexity_sim_10_genera_25_genomes/2021.03.13_09.59.34_sample_0/contigs/anonymous_gsa_filtered.fasta.gz' mode='w' encoding='UTF-8'

In [454]:
for SIM_FASTA_FILE in glob.glob('example_input_data/new_simulations/complexity_sim*/*sample_0*'):
    print('Processing:', SIM_FASTA_FILE)
    minimap2_input = f"{SIM_FASTA_FILE}/reads/anonymous_reads.fq.gz"
    minimap2_output = f"{SIM_FASTA_FILE}/assemblies/re_mapped.bam"

    minimap_index_location = f"{SIM_FASTA_FILE}/catalogue.mmi"
    
    output_filtered_contigs_fasta = os.path.join(BASE_DIR, f"{SIM_FASTA_FILE}/contigs/anonymous_gsa_filtered.fasta.gz")

    !~/anaconda3/envs/vamb_env/bin/minimap2 -d $minimap_index_location $output_filtered_contigs_fasta 
    
    print('minimap index location', minimap_index_location)

    if not os.path.exists(os.path.join(SIM_FASTA_FILE,'assemblies')):
        os.mkdir(os.path.join(SIM_FASTA_FILE,'assemblies'))

    !~/anaconda3/envs/vamb_env/bin/minimap2 \
        -t 10 \
        -N 50 \
        -ax sr $minimap_index_location \
        $minimap2_input | samtools view -F 3584 -b --threads 8 > $minimap2_output

Processing: example_input_data/new_simulations/complexity_sim_1_genera_250_genomes/2021.03.13_09.38.14_sample_0
[M::mm_idx_gen::0.186*1.08] collected minimizers
[M::mm_idx_gen::0.236*1.48] sorted minimizers
[M::main::0.323*1.35] loaded/built the index for 583 target sequence(s)
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 583
[M::mm_idx_stat::0.338*1.33] distinct minimizers: 1000598 (89.04% are singletons); average occurrences: 1.137; average spacing: 5.361
[M::main] Version: 2.17-r941
[M::main] CMD: /Users/pmccaffrey/anaconda3/envs/vamb_env/bin/minimap2 -d example_input_data/new_simulations/complexity_sim_1_genera_250_genomes/2021.03.13_09.38.14_sample_0/catalogue.mmi /Users/pmccaffrey/jupyter_projects/CS230_Stanford_Project/example_input_data/new_simulations/complexity_sim_1_genera_250_genomes/2021.03.13_09.38.14_sample_0/contigs/anonymous_gsa_filtered.fasta.gz
[M::main] Real time: 0.351 sec; CPU: 0.462 sec; Peak RSS: 0.081 GB
minimap index location example_input_data/n

And we calculate the TNF and RPKM matrices

In [455]:
for SIM_FASTA_FILE in glob.glob('example_input_data/new_simulations/complexity_sim*/*sample_0*'):
    output_filtered_contigs_fasta = f"{SIM_FASTA_FILE}/contigs/anonymous_gsa_filtered.fasta.gz"
    
    # Use Reader to open plain or zipped files. File must be opened in binary mode
    with vamb.vambtools.Reader(output_filtered_contigs_fasta, 'rb') as inputfile:
        tnfs, contignames, lengths = vamb.parsecontigs.read_contigs(inputfile)
        
    sample_bamfile = glob.glob(f"{SIM_FASTA_FILE}/assemblies/re_mapped.bam")
    print("bamfile for rpkm:", sample_bamfile)
    
    rpkms = vamb.parsebam.read_bamfiles(sample_bamfile) 
    print('Type of rpkms:', type(rpkms), 'of dtype', rpkms.dtype)
    print('Shape of rpkms', rpkms.shape)
    
    print("tnfs shape:", tnfs.shape)
    
    
    vamb_inputs_base = os.path.join(SIM_FASTA_FILE,'vamb_inputs')
    
    
    if not os.path.exists(vamb_inputs_base):
        os.mkdir(vamb_inputs_base)

    with open(os.path.join(vamb_inputs_base, 'contignames.npz'), 'wb') as file:
        vamb.vambtools.write_npz(file, np.array(contignames))

    with open(os.path.join(vamb_inputs_base, 'lengths.npz'), 'wb') as file:
        vamb.vambtools.write_npz(file, lengths)

    with open(os.path.join(vamb_inputs_base, 'tnfs.npz'), 'wb') as file:
        vamb.vambtools.write_npz(file, tnfs)

    with open(os.path.join(vamb_inputs_base, 'rpkms.npz'), 'wb') as file:
        vamb.vambtools.write_npz(file, rpkms)

bamfile for rpkm: ['example_input_data/new_simulations/complexity_sim_1_genera_250_genomes/2021.03.13_09.38.14_sample_0/assemblies/re_mapped.bam']
Type of rpkms: <class 'numpy.ndarray'> of dtype float32
Shape of rpkms (583, 1)
tnfs shape: (583, 103)
bamfile for rpkm: ['example_input_data/new_simulations/complexity_sim_1_genera_25_genomes/2021.03.13_09.21.48_sample_0/assemblies/re_mapped.bam']
Type of rpkms: <class 'numpy.ndarray'> of dtype float32
Shape of rpkms (2095, 1)
tnfs shape: (2095, 103)
bamfile for rpkm: ['example_input_data/new_simulations/complexity_sim_5_genera_25_genomes/2021.03.13_09.51.05_sample_0/assemblies/re_mapped.bam']
Type of rpkms: <class 'numpy.ndarray'> of dtype float32
Shape of rpkms (2071, 1)
tnfs shape: (2071, 103)
bamfile for rpkm: ['example_input_data/new_simulations/complexity_sim_10_genera_25_genomes/2021.03.13_09.59.34_sample_0/assemblies/re_mapped.bam']
Type of rpkms: <class 'numpy.ndarray'> of dtype float32
Shape of rpkms (1977, 1)
tnfs shape: (1977, 1

In [411]:
f1, f2 = np.random.choice(glob.glob('example_input_data/new_simulations/complexity_sim*/*sample_0*/*assem*/*re_mapped*'), 2)

print(f1 == f2)

!diff $f1 $f2

False
Binary files example_input_data/new_simulations/complexity_sim_10_genera_50_genomes/2021.03.13_10.16.05_sample_0/assemblies/re_mapped.bam and example_input_data/new_simulations/complexity_sim_1_genera_250_genomes/2021.03.13_09.38.14_sample_0/assemblies/re_mapped.bam differ
